# Datenaustausch WIAG/FactGrid, Teil 1
## Lies FactGrid-IDs in WIAG ein.

Wenn IDs im FactGrid manuell recherchiert worden sind und in WIAG eingetragen wurden, dann ist zu prüfen, ob zunächst die entsprechenden WIAG-IDs auch manuell in FactGrid eingetragen werden sollen.

- Lies die aktuellen FQ-IDs aus WIAG ein. 
- Lies die aktuellen WIAG-IDs aus FactGrid ein.
- Prüfe in Bezug auf die WIAG-ID auf doppelte Einträge in WIAG, in FactGrid und auf unterschiedliche Zuordnungen in WIAG und FactGrid
- Entferne Einträge, die im FactGrid schon eine ID haben.

In [82]:
using DataFrames, CSV, HTTP, CodecZlib, Dates

In [83]:
input_path = "C:\\Users\\khan32\\Documents\\factgrid-test-import\\data\\FactGrid"

"C:\\Users\\khan32\\Documents\\factgrid-test-import\\data\\FactGrid"

In [84]:
output_path = "C:\\Users\\khan32\\Documents\\factgrid-test-import\\data_sql"

"C:\\Users\\khan32\\Documents\\factgrid-test-import\\data_sql"

## Lies und prüfe die IDs aus FactGrid

Frage IDs im FactGrid ab (SPARQL) und speichere das Ergebnis in einer `*.tsv`-Datei.  
https://database.factgrid.de/query/


``` sparql
SELECT DISTINCT ?item ?itemLabel ?wiag
	WHERE
	{
	  ?item wdt:P601 ?wiag.
	  ?item wdt:P2 wd:Q7
	    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
	}    

```

Download a csv file using the query given.

In [85]:
r = HTTP.request("GET", "https://database.factgrid.de/sparql", 
    Dict("Accept" => "text/csv"); 
    query=Dict("query" => """SELECT DISTINCT ?item ?itemLabel ?wiag
    WHERE
    {
      ?item wdt:P601 ?wiag.
      ?item wdt:P2 wd:Q7
        SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }    
"""))

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Date: Wed, 24 Apr 2024 11:45:11 GMT
Server: Jetty(9.4.12.v20180830)
Content-Type: text/csv;charset=utf-8
Content-disposition: attachment; filename=query35559.csv
X-FIRST-SOLUTION-MILLIS: 8
Access-Control-Allow-Origin: *
Transfer-Encoding: chunked

item,itemLabel,wiag
https://database.factgrid.de/entity/Q77343,Georg Friedrich Theodor von Witzendorff,WIAG-Pers-CANON-26965-001
https://database.factgrid.de/entity/Q102444,Christof von Kesselstatt,WIAG-Pers-CANON-10981-001
https://database.factgrid.de/entity/Q140745,Detlev Joachim von Brockdorff,WIAG-Pers-CANON-26937-001
https://database.factgrid.de/entity/Q140685,Friedrich Carl Schnoor,WIAG-Pers-CANON-27011-001
https://database.factgrid.de/entity/Q140710,Friedrich Ludwig von Moltke,WIAG-Pers-CANON-26943-001
https://database.factgrid.de/entity/Q144133,Joseph Anton Siegmund von Beroldingen,WIAG-Pers-CANON-10894-001
https://database.factgrid.de/entity/Q146400,Franz Paul Christoph Freiherr von Seckend

Save the downloaded csv file.

In [86]:
tsv_file = "FactGrid-IDs_" * Dates.format(now(), "yyyy-mm-dd-HH-MM-SS") * ".csv"
tsv_file_path = joinpath(input_path, tsv_file)
open(tsv_file_path, "w") do out
    println(out, String(r.body))
end

Lies die Daten ein. Gib die Zahl der Einträge aus.

In [87]:
#input_file = "FactGrid-IDs_2024-03-26_1.tsv"
input_file = tsv_file
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\khan32\\Documents\\factgrid-test-import\\data\\FactGrid\\FactGrid-IDs_2024-04-24-13-45-19.csv"

In [88]:
#df_fq_ids = CSV.read(input_path_file, DataFrame, delim = "\t");
df_fq_ids = CSV.read(input_path_file, DataFrame);
nrow(df_fq_ids)

11660

Kontroll-Ausgabe

In [89]:
df_fq_ids[17:22, :]

Row,item,itemLabel,wiag
,String,String,String31
1,https://database.factgrid.de/entity/Q254428,Arnold von Rotberg,WIAG-Pers-EPISCGatz-05560-001
2,https://database.factgrid.de/entity/Q254416,Berthold von Landsberg,WIAG-Pers-EPISCGatz-05470-001
3,https://database.factgrid.de/entity/Q254424,Dietrich Schenk von Erbach,WIAG-Pers-EPISCGatz-05586-001
4,https://database.factgrid.de/entity/Q254420,Johann Borsnitz,WIAG-Pers-EPISCGatz-03013-001
5,https://database.factgrid.de/entity/Q254601,Friedrich zu Rhein,WIAG-Pers-EPISCGatz-05679-001
6,https://database.factgrid.de/entity/Q254922,Burkhard von Warberg,WIAG-Pers-EPISCGatz-05655-001


Extrahiere die Q-Nummer aus der Spalte `item`.

In [90]:
find_fq_id(s) = last(split(s, "/"))

find_fq_id (generic function with 1 method)

In [91]:
transform!(df_fq_ids, :item => ByRow(find_fq_id) => :q_id);

**Finde doppelte Einträge in FactGrid**

In [92]:
df_fq_m = groupby(df_fq_ids, :wiag);
size(df_fq_m)

(11660,)

In [93]:
df_fq_m_count = combine(df_fq_m, nrow => :n);

Zeige, WIAG-Nummern, die in FactGrid für mehrere Items verwendet werden.

In [94]:
df_fq_m_2 = subset(df_fq_m_count, :n => ByRow(n -> n > 1))

Row,wiag,n
,String31,Int64


Falls es Mehrfacheinträge gibt: Erhalte mehr Informationen durch einen Filter für diese Einträge. Die unten angegebenen WIAG-Nummern sind Beispiele, welche durch die WIAG-Nummern der vorangegangenen, aktuellen Ausgabe ersetzt werden müssen.

In [95]:
wiag_mult = [
    "WIAG-Pers-CANON-11582-001", 
    "WIAG-Pers-CANON-15754-001"
]

2-element Vector{String}:
 "WIAG-Pers-CANON-11582-001"
 "WIAG-Pers-CANON-15754-001"

In [96]:
crit_wiag(s) = s in wiag_mult
subset(df_fq_ids, :wiag => ByRow(crit_wiag))

Row,item,itemLabel,wiag,q_id
,String,String,String31,SubStrin…
1,https://database.factgrid.de/entity/Q509010,Hermann of Baden-Baden,WIAG-Pers-CANON-11582-001,Q509010
2,https://database.factgrid.de/entity/Q509641,"Francis Christopher Anton, Count of Hohenzollern-Sigmaringen",WIAG-Pers-CANON-15754-001,Q509641


Falls es Mehrfacheinträge gibt, müssen diese redaktionell in FactGrid geprüft werden. Ggf. müssen dort Dubletten zusammengeführt werden; FactGrid-IDs, die dann nicht mehr existieren, werden in die folgende Filterliste eingetragen. Die beiden schon eingetragenen Beispiel-Q-IDs dafür überschrieben. Die Filterliste enthält Q-IDs, die **nicht** in WIAG übernommen werden sollen.

In [97]:
crit_fq(s) = s in ["Q646386", "Q646394"]

crit_fq (generic function with 1 method)

In [98]:
df_fq_ids = subset(df_fq_ids, :q_id => ByRow(!crit_fq));
nrow(df_fq_ids)

11660

Die Ausgabe der folgenden Zelle sollte keine Mehrfacheinträge mehr enthalten.

In [99]:
subset(df_fq_ids, :wiag => ByRow(crit_wiag))

Row,item,itemLabel,wiag,q_id
,String,String,String31,SubStrin…
1,https://database.factgrid.de/entity/Q509010,Hermann of Baden-Baden,WIAG-Pers-CANON-11582-001,Q509010
2,https://database.factgrid.de/entity/Q509641,"Francis Christopher Anton, Count of Hohenzollern-Sigmaringen",WIAG-Pers-CANON-15754-001,Q509641


## Lies und prüfe die FactGrid-IDs aus WIAG

Exportiere Personendaten aus WIAG (https://wiag-vocab.adw-goe.de): Export-CSV Personendaten, speichere die Datei im Input-Ordner (siehe oben) unter dem Dateinamen WIAG-can.csv. Die Datei kann auch anders benannt werden, dann muss auch der Eintrag in der Zeile input_file geändert werden 
Letzer Export: Server LIVE, Datum 2023-12-18 12:20

Lies die Personendaten aus WIAG ein.

In [100]:
# r = HTTP.request("POST", "http://wiag-vocab.adw-goe.de/download/csv/person/can", 
#     Dict("Accept" => "text/csv"), 
#     HTTP.Form(Dict(
#         "person_form[name]" => "",
#         "person_form[domstift]"=> "",
#         "person_form[office]"=> "",
#         "person_form[place]"=> "",
#         "person_form[year]"=> "",
#         "person_form[someid]"=> "",
#         "person_form[sortBy]"=> "domstift",
#         "person_form[sortOrder]"=> "ASC",
#         "person_form[corpus]"=> "can",
#         "pageNumber"=> ["1","1"],
#         "person_form[stateFctCap]"=> "",
#         "person_form[stateFctOfc]"=> "",
#         "person_form[stateFctPlc]"=> "",
#         "person_form[stateFctUrl]"=> "",
#         "person_form[stateFctDioc]"=> "",
#         "person_form[_token]"=> "5cbea5f890d070bbb4f0350f.FwxBYKsOwnzbM_dAhHfyOT2s6TzCBu2sMud65S3ufO8.fWMoDeFCs0vrdaZ09g63aVT9k3OIPonGR5c5rEvXO91-Si0yynyKGJljkA"
#     )); 
#     query=Dict("query" => """SELECT DISTINCT ?item ?itemLabel ?wiag
#     WHERE
#     {
#       ?item wdt:P601 ?wiag.
#       ?item wdt:P2 wd:Q7
#         SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
#     }    
# """))

In [101]:
input_file = "WIAG-can_2024_04_24_whole.csv"
input_path_file = joinpath(input_path, input_file)

"C:\\Users\\khan32\\Documents\\factgrid-test-import\\data\\FactGrid\\WIAG-can_2024_04_24_whole.csv"

In [102]:
df_p = CSV.read(input_path_file, DataFrame);
nrow(df_p)

15503

Wähle relevante Spalten aus.

In [103]:
df_p_fq = select(df_p, [:id, :FactGrid_ID, :corpus, :displayname]);

## Vergleiche die beiden Quellen

In [104]:
df_fq = outerjoin(df_p_fq, df_fq_ids, on = :id => :wiag);
nrow(df_fq)

19594

Die Zahl ist größer als die Zahl der Einträge in `df_p_fq` aus WIAG, weil in der Liste aus FactGrid die Bischöfe auch enthalten sind.

In [105]:
names(df_fq)

7-element Vector{String}:
 "id"
 "FactGrid_ID"
 "corpus"
 "displayname"
 "item"
 "itemLabel"
 "q_id"

Ermittle die Einträge mit FactGrid-ID in WIAG aber ohne Eintrag in FactGrid.

In [106]:
a_mg_b(a, b) = !ismissing(a) && ismissing(b)

a_mg_b (generic function with 1 method)

In [107]:
df_p_only = subset(df_fq, [:FactGrid_ID, :q_id] => ByRow(a_mg_b))

Row,id,FactGrid_ID,corpus,displayname,item,itemLabel,q_id
,String31,String7?,String15?,String?,String?,String?,SubStrin…?
1,WIAG-Pers-CANON-13861-001,Q701486,can,Anton Eusebius Graf von Königsegg-Aulendorf (Ober-)Staufen und Ebenweiler,missing,missing,missing


Ermittle die Einträge mit unterschiedlichen Werten für die FactGrid-ID

In [108]:
missing_or_equal(a, b) = ismissing(a) || ismissing(b) || a == b

missing_or_equal (generic function with 1 method)

In [109]:
df_diff = subset(df_fq, [:FactGrid_ID, :q_id] => ByRow(!missing_or_equal))

Row,id,FactGrid_ID,corpus,displayname,item,itemLabel,q_id
,String31,String7?,String15?,String?,String?,String?,SubStrin…?
1,WIAG-Pers-CANON-49064-001,Q701452,can,Johann von Schaumberg,https://database.factgrid.de/entity/Q883556,Johann von Schaumberg,Q883556
2,WIAG-Pers-CANON-25919-001,Q883726,can,Valentin Philipp von Nankenreuth,https://database.factgrid.de/entity/Q729654,Valentin Philipp von Nankenreuth,Q729654
3,WIAG-Pers-CANON-19541-001,Q726649,can,Johannes Langeditrich,https://database.factgrid.de/entity/Q480961,Johannes Lange Diderick,Q480961


Sortiere die Bischöfe wieder aus. (Nach der Prüfung auf ungleiche IDs!) 

In [110]:
is_gatz_id(s) = occursin("EPISCGatz", s)

is_gatz_id (generic function with 1 method)

In [111]:
subset!(df_fq, :id => ByRow(!is_gatz_id))
nrow(df_fq)

14429

Zahl der neu einzutragenden FactGrid-IDs: Einträge mit einer ID im FactGrid, aber nicht in WIAG

In [112]:
subset(df_fq, [:q_id, :FactGrid_ID] => ByRow(a_mg_b))

Row,id,FactGrid_ID,corpus,displayname,item,itemLabel,q_id
,String31,String7?,String15?,String?,String?,String?,SubStrin…?
1,WIAG-Pers-CANON-48917-001,missing,can,Sigismund von Leonrod,https://database.factgrid.de/entity/Q900400,Sigismund von Leonrod,Q900400
2,WIAG-Pers-CANON-48966-001,missing,can,Veit Truchseß von Pommersfelden,https://database.factgrid.de/entity/Q900419,Veit Truchseß von Pommersfelden,Q900419
3,WIAG-Pers-CANON-49026-001,missing,can,Gottfried von Wolfstein,https://database.factgrid.de/entity/Q900437,Gottfried von Wolfstein,Q900437
4,WIAG-Pers-CANON-49085-001,missing,can,Peter von Pienzenau,https://database.factgrid.de/entity/Q900454,Peter von Pienzenau,Q900454
5,WIAG-Pers-CANON-48815-001,missing,can,Heinrich Marschalk von Ebneth,https://database.factgrid.de/entity/Q900353,Heinrich Marschalk von Ebneth,Q900353
6,WIAG-Pers-CANON-48816-001,missing,can,Konrad von Stör,https://database.factgrid.de/entity/Q900354,Konrad von Stör,Q900354
7,WIAG-Pers-CANON-48840-001,missing,can,Werntho von Breitenstein,https://database.factgrid.de/entity/Q900365,Werntho von Breitenstein,Q900365
8,WIAG-Pers-CANON-48832-001,missing,can,Otto von Egloffstein,https://database.factgrid.de/entity/Q900360,Otto von Egloffstein,Q900360
9,WIAG-Pers-CANON-48833-001,missing,can,Hermann von Henneberg,https://database.factgrid.de/entity/Q900361,Hermann von Henneberg,Q900361


In [113]:
df_insert = subset(df_fq, [:q_id, :FactGrid_ID] => ByRow(a_mg_b));
macro assert(ex)
   return :( $ex ? nothing : throw(AssertionError($(string(ex)))) )
end
nrow(df_insert)
@assert nrow(df_insert) < 500

Beispiel für das SQL-insert Statement, das in der Folge automatisch für jede Zeile in `df_insert` erzeugt wird.
``` sql
INSERT INTO url_external (item_id, value, authority_id)
	    SELECT item_id, 'Q648993', 42 FROM item_corpus
	    WHERE id_public = "WIAG-Pers-EPISCGatz-02525-001";
```

In [114]:
function make_insert_stmt(fq_id, wiag_id)
    return """
    INSERT INTO url_external (item_id, value, authority_id)
    SELECT item_id, '$(fq_id)', 42 FROM item_corpus
    WHERE id_public = "$(wiag_id)";
    """
end

make_insert_stmt (generic function with 1 method)

Schreibe eine Datei mit SQL-Statements, mit der die FactGrid IDs in die Tabelle `url_external` eingelesen werden können.

In [115]:
output_file = "insert-uext-can_2024_04_24.sql"
output_file_path = joinpath(output_path, output_file)

"C:\\Users\\khan32\\Documents\\factgrid-test-import\\data_sql\\insert-uext-can_2024_04_24.sql"

In [116]:
open(output_file_path, "w") do out
    println(out, "LOCK TABLES url_external WRITE, item_corpus WRITE;")
    for row in eachrow(df_insert)
        println(out, make_insert_stmt(row[:q_id], row[:id]))
    end
    println(out, "UNLOCK TABLES;")
end


Lies die entstandene Datei in die WIAG-Datenbank ein. Die Datei kann z.B. über phpMyAdmin in eine Datenbank eingelesen werden. Nachdem die Datenbank dort ausgewählt ist, wählt man im Menu den Punkt "Import", setzt die Datei ein über "Durchsuchen" und startet den Import mit "Ok".